# Python

![](https://leukipp.github.io/xmastree/kernelspecs/python.png)

In [ ]:
import os
import js
import csv
import urllib
import numpy as np

from pyodide_js import FS
from IPython.display import HTML, Javascript, display


## Utils

In [ ]:
class FileSystemUtils(object):
    def __init__(self, fs, path):
        """ indexedDB filesystem storage """
        self.fs = fs
        self.path = path
        self.type = self.fs.filesystems.IDBFS

        self.mount()
        self.upsync()

    def mount(self):
        """ mount kernel filesystem """
        if not os.path.exists(self.path):
            self.fs.mkdir(self.path)
            self.fs.mount(self.type, {}, self.path)

    def upsync(self):
        """ sync from browser to kernel filesystem """
        self.fs.syncfs(True, lambda x: print(x) if x else None)

    def downsync(self):
        """ sync from kernel to browser filesystem """
        self.fs.syncfs(False, lambda x: print(x) if x else None)


class Simulator(object):
    def __init__(self, width='100%', height='600px'):
        """ three js simulator """
        self.width = width
        self.height = height
        self.settings = dict()
        self.rendered = False

        self.id = 'simulator'
        self.root = '' if '127.0.0.1' in str(js.location) else '/xmastree'
        self.url = f'{self.root}/files/simulator/index.html'

    def src(self):
        """ source url for iframe """
        return f'{self.url}#{urllib.parse.urlencode(self.settings)}'

    def render(self, **kwargs):
        """ render simulator html """
        self.rendered = False
        self.update(**kwargs)
        self.rendered = True
        layout = {
            'id': self.id,
            'frameborder': '0',
            'width': self.width,
            'height': self.height,
            'src': self.src()
        }
        attributes = ' '.join([f'{k}="{v}"' for k, v in layout.items()])
        display(HTML(f'<iframe allowfullscreen {attributes} />'))

    def update(self, **kwargs):
        """ update simulator settings """
        for k, v in kwargs.items():
            self.settings[k] = str(v)
        self.settings['seed'] = int(np.random.rand() * 1e6)
        if self.rendered:
            display(Javascript(f'document.getElementById("{self.id}").src="{self.src()}"'))


class Frames(list):
    def __init__(self, coords_csv, fps=30):
        """ animation frames helper """
        coords = list(csv.reader(coords_csv.split('\n')))
        self.coords = np.array([[float(col) for col in row] for row in coords])
        self.size = self.coords.shape[0]
        self.fps = fps

    def export(self, name, fs):
        """ export frames as csv """
        head = ['FRAME_ID'] + np.hstack([[f'R_{i}', f'G_{i}', f'B_{i}'] for i in range(self.size)]).tolist()
        rows = np.array(self, dtype=int)

        ids = np.arange(rows.shape[0]).reshape(-1, 1)
        frames = np.reshape(rows, (*rows.shape[:1], -1))
        data = np.hstack([ids, frames])

        path = os.path.join(fs.path, name)
        np.savetxt(path, data, header=','.join(head), fmt='%i', delimiter=',', comments='')
        fs.downsync()

        return path


## Setup

In [ ]:
# init filesystem
fs = FileSystemUtils(FS, path=os.path.join(os.path.expanduser('~'), 'data'))

# init simulator
simulator = Simulator()

# init frames
response = await js.fetch('https://raw.githubusercontent.com/standupmaths/xmastree2021/main/coords_2021.csv')
coords = await response.text()
frames = Frames(coords, fps=30)


## Code

### Solar eclipse of December 4, 2021

- `Weather assumptions`: clear sky
- `Maximum eclipse duration`: 114 seconds

[wikipedia.org/wiki/Solar_eclipse_of_December_4,_2021](https://en.wikipedia.org/wiki/Solar_eclipse_of_December_4,_2021)

| [Event](https://www.youtube.com/watch?v=J04GFN2Pq1w) | Time                |
| ---------------------------------------------------- | ------------------- |
| Partial Eclipse Start                                | 2021-12-04 06:41:02 |
| Total Eclipse Start                                  | 2021-12-04 07:32:24 |
| Maximum Eclipse                                      | 2021-12-04 07:33:20 |
| Total Eclipse End                                    | 2021-12-04 07:34:15 |
| Partial Eclipse End                                  | 2021-12-04 08:26:34 |

In [ ]:
# clear frames
frames.clear()


# ------------------------------------------ START CODING ------------------------------------------


def sphere(frames, radius, center, color_in, color_out):
    """ helper function to obtain sphere colored frames """

    # [A] turn all lights on
    frame = np.tile(color_in, (frames.size, 1))

    # [B] mask lights out of sphere
    dist = np.linalg.norm(np.array(center) - frames.coords, axis=1)
    frame[dist > radius] = color_out

    return frame


# define the run time (60s seconds ~ 100x faster than real)
end_time = 60 * frames.fps
center_time = end_time / 2
totality_time = 2 * frames.fps

# write frames
for t in np.arange(end_time):

    # [1] earth shadow moving in front of sun (actually a solid object crashes right into the sun)
    shadow_radius = 1.0
    shadow_offset = np.interp(t, [0, center_time - totality_time / 2], [0.0, 2 * shadow_radius])
    shadow_offset += np.interp(t, [center_time + totality_time / 2, end_time], [0.0, 2 * shadow_radius])
    shadow_center = np.array([0, -2 * shadow_radius + shadow_offset, 1.0])
    shadow_frame = sphere(frames, shadow_radius, shadow_center, [1, 1, 1], [0, 0, 0])

    # [2] sun parameters (sphere is in the center of the tree)
    sun_radius = 1.0
    sun_center = [0, 0, 1.0]
    sun_color_in = [55, 50, 5]
    sun_color_out = [10, 10, 10]

    # [3] sun during totality (corona / diamond ring effect)
    if np.isclose(shadow_center[1], sun_center[1]):
        tt = np.abs(center_time - totality_time / 2 - t)
        sun_corona = np.interp(tt, [10, totality_time / 2], [1.0, 1.1], 0, 0)
        sun_corona += np.interp(tt, [1 + totality_time / 2, totality_time - 10], [1.1, 1.0], 0, 0)
        sun_radius *= sun_corona
        sun_color_in = [20, 20, int(15 * sun_corona)]
    sun_frame = sphere(frames, sun_radius, sun_center, sun_color_in, sun_color_out)

    # [4] merge shadow colors with sun colors (overwrite with nonzero values)
    frame = np.where(shadow_frame == 0, sun_frame, shadow_frame)

    # [5] glow effect to brighten lights (if less are on intensive them)
    if not np.isclose(shadow_center[1], sun_center[1]):
        active_mask = frame > 15
        active_count = np.count_nonzero(active_mask) // 3
        frame[active_mask] += np.interp(active_count, [0, 200], [50, 0]).astype(np.uint8)

    # append frame
    frames.append(frame)


# ------------------------------------------- END CODING -------------------------------------------


# export frames
path = frames.export('antarctic_solar_eclipse.csv', fs)

# update simulator
simulator.update(fps=frames.fps, animations=f'fs:/{path}')


In [ ]:
# render simulator
simulator.render(loop='true', rotation=0)
